In [1]:
import json
import pandas as pd
json_file_path = '../results/pubmed_results.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

df = pd.DataFrame.from_dict(loaded_data)

In [2]:
from trulens.providers.openai import OpenAI
from dotenv import load_dotenv
import os

#load_dotenv()
provider = OpenAI()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [3]:
from trulens.apps.virtual import VirtualApp
from trulens.core import Select
from trulens.core import Feedback

virtual_app = dict(
    llm=dict(
        modelname="GPT3.5Turbo"
    ),
    template="information about the template I used in my app",
    debug="all of these fields are completely optional"
)

virtual_app = VirtualApp(virtual_app) # can start with the prior dictionary
virtual_app[Select.RecordCalls.llm.maxtokens] = 1024

retriever_component = Select.RecordCalls.retriever
virtual_app[retriever_component] = "Retriever"

context_call = retriever_component.get_context
context = context_call.rets[:]

In [4]:
len = 20

In [5]:
from trulens.apps.virtual import VirtualRecord

data_dict = df.to_dict('records')

data = []

for record in data_dict[:len]:
    rec = VirtualRecord(
        main_input=record['question'],
        main_output=record['answer'],
        meta=record['id'],
        calls=
            {
                context_call: dict(
                    args=[record['question']],
                    rets=[record['contexts']]
                )
            }
        )
    data.append(rec)

In [6]:
df_modified = df.rename(columns={"question": "query", "ground_truth": "expected_response"})

In [7]:
from trulens.feedback import GroundTruthAgreement

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

#comprehensiveness evaluation in RAG with the retrieved context (i.e. the source) and response (i.e. the summary)
f_comprehensiveness = Feedback(
    provider.comprehensiveness_with_cot_reasons, name="Comprehensiveness"
).on_input_output()


f_groundtruth = Feedback(
    GroundTruthAgreement(df_modified[:len], provider=provider).agreement_measure, name="Ground Truth Eval"
).on_input_output()

✅ In Groundedness, input source will be set to __record__.app.retriever.get_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Comprehensiveness, input source will be set to __record__.main_input or `Select.RecordInput` .
✅ In Comprehensiveness, input summary will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Ground Truth Eval, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth Eval, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [8]:
from trulens.apps.virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_name="pubmedQA-demo2",
    app=virtual_app,
    feedbacks=[f_groundedness]
)
# feedbacks=[f_qa_relevance,f_groundedness,f_groundtruth,f_comprehensiveness]


In [9]:
import time
pace_settings = {
    "seconds_per_period": 30.0,  # Reduce from 60.0 to 30.0 seconds
    "marks_per_second": 0.5       # Reduce from 1.0 to 0.5 marks per second
}
global current_pace_settings
current_pace_settings = pace_settings
i = 1
for record in data:
     # Wait based on the pace settings
    time.sleep(pace_settings["seconds_per_period"] / pace_settings["marks_per_second"])
    print(i,record)
    i = i + 1
    virtual_recorder.add_record(record)

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


1 Record(record_hash_0bfd797f3e68eb8362a72c595a7cf784) with 2 calls:
  RecordAppCall: .root -> app.retriever.get_context
  RecordAppCall: .root



Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


2 Record(record_hash_77ed6903f2fa07f1be4719fc9ce9971c) with 2 calls:
  RecordAppCall: .root -> app.retriever.get_context
  RecordAppCall: .root



Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


...

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


18 Record(record_hash_da38d7ff1ffbe2b6b14f77e13cdf82ab) with 2 calls:
  RecordAppCall: .root -> app.retriever.get_context
  RecordAppCall: .root



Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


19 Record(record_hash_033d74ef0e4cbe52175e68ce4d7cb74b) with 2 calls:
  RecordAppCall: .root -> app.retriever.get_context
  RecordAppCall: .root

20 Record(record_hash_9b79d27f962ebfae122a1cf2480f8847) with 2 calls:
  RecordAppCall: .root -> app.retriever.get_context
  RecordAppCall: .root



Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


In [10]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()

In [11]:
appId=virtual_recorder.app_id
print(appId)
records_df, feedbacks =session.get_records_and_feedback(app_ids=[appId])

app_hash_567b82187ab75369b432d4a48baed4ed


In [12]:
import json

records_df["id"] = records_df["record_json"].apply(lambda x : json.loads(x)['meta'])
records_df = pd.merge(records_df,df_modified,on=['id'])

In [13]:
session.get_leaderboard(app_ids=[appId])

,,Groundedness,latency,total_cost
app_name,app_version,,,
pubmedQA-demo2,base,0.950292,0.000121,0.0


In [14]:
records_df

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,latency,total_tokens,total_cost,cost_currency,id,query,answer,contexts,expected_response,label
0,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_9b79d27f962ebfae122a1cf2480f8847,"""Internal derangement of the temporomandibular...","""Ultrasound demonstrates considerably lower se...",,"{""record_id"": ""record_hash_9b79d27f962ebfae122...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-11-25T02:48:37.791382"", ""...",...,0.000093,0,0.0,USD,22668712,Internal derangement of the temporomandibular ...,Ultrasound demonstrates considerably lower sen...,[The aim of this study was to assess the diagn...,No. The present study does not support the rec...,PASS
1,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_033d74ef0e4cbe52175e68ce4d7cb74b,"""Two-year follow-up survey of patients with al...","""The prognosis of patients with allergic conta...",,"{""record_id"": ""record_hash_033d74ef0e4cbe52175...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-11-25T02:48:37.789361"", ""...",...,0.000091,0,0.0,USD,24359102,Two-year follow-up survey of patients with all...,The prognosis of patients with allergic contac...,[Skin diseases are the most frequently recogni...,"Yes. At follow-up, patients with contact urtic...",PASS
2,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_da38d7ff1ffbe2b6b14f77e13cdf82ab,"""Does the SCL 90-R obsessive-compulsive dimens...","""No, the SCL 90-R obsessive-compulsive subscal...",,"{""record_id"": ""record_hash_da38d7ff1ffbe2b6b14...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-11-25T02:48:37.787322"", ""...",...,0.000097,0,0.0,USD,9582182,Does the SCL 90-R obsessive-compulsive dimensi...,"No, the SCL 90-R obsessive-compulsive subscale...",[To investigate the relevance of the Symptom C...,Yes. Our data suggest that the SCL 90-R is bes...,PASS
3,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_55070a93e96288cfef0539769830570b,"""Does desflurane alter left ventricular functi...","""No change in left ventricular function was no...",,"{""record_id"": ""record_hash_55070a93e96288cfef0...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-11-25T02:48:37.785071"", ""...",...,0.000103,0,0.0,USD,10456814,Does desflurane alter left ventricular functio...,No change in left ventricular function was not...,[Although desflurane is commonly used to contr...,No. This study demonstrates that in patients a...,PASS
4,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_257356189f318b02de84273559c92755,"""Doppler examination of uteroplacental circula...","""Yes, the study found that higher resistance i...",,"{""record_id"": ""record_hash_257356189f318b02de8...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-11-25T02:48:37.782968"", ""...",...,0.000092,0,0.0,USD,17551944,Doppler examination of uteroplacental circulat...,"Yes, the study found that higher resistance in...",[To determine whether spectral Doppler measure...,Yes. Transvaginal Doppler examination can dete...,PASS
5,app_hash_567b82187ab75369b432d4a48baed4ed,"{""tru_class_info"": {""name"": ""TruVirtual"", ""mod...",VirtualApp(trulens.apps.virtual),record_hash_108ffea4873a012b713d3e975349f5b8,"""Is fetal anatomic assessment on follow-up ant...","""Yes, fetal anatomic assessment on follow-up a...",,"{""record_id"": ""record_hash_108ffea4873a012b713...","{""n_